In [ ]:
#initial DE component
import numpy as np
import tensorflow as tf

# Load trained deep learning models
base_m1 = tf.keras.models.load_model("model_1.h5")
base_m2 = tf.keras.models.load_model("model_2.h5")
base_m3 = tf.keras.models.load_model("model_3.h5")

# Load validation data
X_val = np.load("X_val.npy")
y_val = np.load("y_val.npy")

# Get model predictions on validation data
preds_1 = base_m1.predict(X_val)
preds_2 = base_m2.predict(X_val)
preds_3 = base_m3.predict(X_val)

# Stack predictions (num_models, num_samples, num_classes)
predictions = np.array([preds_1, preds_2, preds_3])
num_models, num_samples, num_classes = predictions.shape

# Flatten predictions for easier computation
predictions = predictions.reshape(num_models, -1)

# DE hyperparameters
POP_SIZE = 20   # Number of candidate weight vectors
F = 0.6         # Scaling factor for mutation (typically 0.4 - 1.2)
CR = 0.9        # Crossover probability
MAX_ITER = 100  # Maximum iterations

# Initialize a population of weight vectors (each weight vector sums to 1)
population = np.random.dirichlet(np.ones(num_models), size=POP_SIZE)


In [ ]:
#mutation function
def mutate(population, F):
    """
    Perform mutation in Differential Evolution (DE).

    - population: (POP_SIZE, num_models) matrix of weight vectors.
    - F: Scaling factor for mutation.

    Returns:
    - mutant_vectors: New mutated weight vectors.
    """
    pop_size, dimensions = population.shape
    mutant_vectors = np.zeros((pop_size, dimensions))

    for i in range(pop_size):
        # Select 3 random distinct indices r1, r2, r3 different from i
        indices = np.random.choice([j for j in range(pop_size) if j != i], 3, replace=False)
        r1, r2, r3 = indices

        # Compute mutant vector using DE formula: Vi = Xr1 + F * (Xr2 - Xr3)
        mutant_vector = population[r1] + F * (population[r2] - population[r3])

        # Ensure weights are between 0 and 1, and normalize to sum to 1
        mutant_vector = np.clip(mutant_vector, 0, 1)
        mutant_vector /= np.sum(mutant_vector)

        mutant_vectors[i] = mutant_vector

    return mutant_vectors


In [ ]:
#crossover function
def crossover(mutants, population, CR):
    """
    Perform crossover in DE to create trial vectors.

    - mutants: Mutated population.
    - population: Original population.
    - CR: Crossover probability.

    Returns:
    - trial_vectors: Offspring vectors after crossover.
    """
    pop_size, dimensions = population.shape
    trial_vectors = np.zeros((pop_size, dimensions))

    for i in range(pop_size):
        # Perform binomial crossover
        crossover_mask = np.random.rand(dimensions) < CR  # Boolean mask
        trial_vector = np.where(crossover_mask, mutants[i], population[i])

        # Ensure the new weight vector remains valid
        trial_vector = np.clip(trial_vector, 0, 1)
        trial_vector /= np.sum(trial_vector)

        trial_vectors[i] = trial_vector

    return trial_vectors


In [ ]:
#define loss
def evaluate(weights):
    """
    Compute the loss (categorical cross-entropy) for a given weight vector.

    - weights: Weight vector for combining models.

    Returns:
    - Loss value (lower is better).
    """
    weighted_preds = np.dot(weights, predictions).reshape(num_samples, num_classes)  # Weighted sum
    loss = tf.keras.losses.categorical_crossentropy(y_val, weighted_preds).numpy()
    return np.mean(loss)  # Return mean loss


In [ ]:
#optimization
best_weights = None
best_loss = float("inf")

for iteration in range(MAX_ITER):
    # Step 1: Mutation
    mutants = mutate(population, F)

    # Step 2: Crossover
    trial_vectors = crossover(mutants, population, CR)

    # Step 3: Selection
    for i in range(POP_SIZE):
        trial_loss = evaluate(trial_vectors[i])
        original_loss = evaluate(population[i])

        if trial_loss < original_loss:  # Select the better weight vector
            population[i] = trial_vectors[i]

            # Track the best solution
            if trial_loss < best_loss:
                best_loss = trial_loss
                best_weights = trial_vectors[i]

    # Print progress
    print(f"Iteration {iteration + 1}/{MAX_ITER} - Best Loss: {best_loss:.5f}")

# Final optimized weights
print("Optimized Weights:", best_weights)


In [ ]:
# Compute ensemble predictions with optimized weights
final_predictions = np.dot(best_weights, predictions).reshape(num_samples, num_classes)

# Convert predictions to class labels
final_labels = np.argmax(final_predictions, axis=1)
true_labels = np.argmax(y_val, axis=1)

# Compute accuracy
accuracy = np.mean(final_labels == true_labels)
print("Final Ensemble Accuracy:", accuracy)
